# `Homework_Chapter 2: Regression`

### Set up

Select the option that's closest to your solution.

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Getting the data

For this homework, we'll use the California Housing Prices dataset. Download it from
[here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv).

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2023-09-27 15:43:56--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.07s   

2023-09-27 15:43:57 (19.8 MB/s) - ‘housing.csv’ saved [1423529/1423529]



### Preparing the dataset

For this homework, we only want to use a subset of data.

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Next, use only the following columns:

* `'latitude'`,
* `'longitude'`,
* `'housing_median_age'`,
* `'total_rooms'`,
* `'total_bedrooms'`,
* `'population'`,
* `'households'`,
* `'median_income'`,
* `'median_house_value'`

### Question 1

There's one feature with missing values. What is it?

* `total_rooms`
* `total_bedrooms`
* `population`
* `households`

In [3]:
df_housing = pd.read_csv('housing.csv')
df_housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [4]:
df_housing['ocean_proximity'].unique()
df_housing = df_housing[(df_housing['ocean_proximity'] == '<1H OCEAN') | (df_housing['ocean_proximity'] == 'INLAND')]

df_housing = df_housing.drop('ocean_proximity', axis=1)
df_housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0


In [5]:
df_housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15687 entries, 701 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           15687 non-null  float64
 1   latitude            15687 non-null  float64
 2   housing_median_age  15687 non-null  float64
 3   total_rooms         15687 non-null  float64
 4   total_bedrooms      15530 non-null  float64
 5   population          15687 non-null  float64
 6   households          15687 non-null  float64
 7   median_income       15687 non-null  float64
 8   median_house_value  15687 non-null  float64
dtypes: float64(9)
memory usage: 1.2 MB


In [6]:
df_housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [7]:
print(f"Columns with missing values: {df_housing.columns[df_housing.isnull().any()].tolist()}")

Columns with missing values: ['total_bedrooms']


### Question 2

What's the median (50% percentile) for variable `'population'`?

- 995
- 1095
- 1195
- 1295

In [8]:
df_housing["population"].quantile(0.5)

1195.0

In [9]:
print(f"The 50th percentile of population is: {df_housing['population'].quantile(0.5)}")

The 50th percentile of population is: 1195.0


### Prepare and split the dataset

* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Apply the log transformation to the `median_house_value` variable using the `np.log1p()` function.


In [10]:
# Distributions of data
n = len(df_housing)

n_train = int(0.6*n)
n_val = int(0.2*n)
#n_test = int(0.2*n)

In [11]:
# Shuffle data
np.random.seed(42)
idx = np.arange(n)
np.random.shuffle(idx)
df_housing_shuffled = df_housing.iloc[idx]

In [12]:
# Split data
X_train = df_housing_shuffled.iloc[:n_train].copy()
X_val =  df_housing_shuffled.iloc[n_train:n_train + n_val].copy()
X_test = df_housing_shuffled.iloc[n_train+n_val:].copy()

In [13]:
# Apply log transformation
Y_train = np.log1p( X_train['median_house_value'] ).values
Y_val = np.log1p( X_val['median_house_value'] ).values
Y_test = np.log1p( X_test['median_house_value']).values

In [14]:
del X_train['median_house_value']
del X_val['median_house_value']
del X_test['median_house_value']

In [15]:
def linear_regression(X, y):
    # adding ones in the dataset X
    X_0 = np.ones(X.shape[0])
    X = np.column_stack([X_0, X])

    XTX = X.T.dot(X)
    XTX_inverse = np.linalg.inv(XTX)
    w = XTX_inverse.dot(X.T).dot(y)

    Y_pred = X.dot(w)
    return Y_pred

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:

- With 0
- With mean
- Both are equally good

In [16]:
# filling mean and zeros missing values
def handle_nan(df, feature, fillnan_with):
    df_copy = df.copy()
    if fillnan_with == 'mean':
        df_copy[feature].fillna(value = df_copy[feature].mean(), inplace=True)
    elif fillnan_with == 'zero':
        df_copy[feature].fillna(value = 0, inplace=True)

    return df_copy.values

In [17]:
# Root mean squared error
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    rmse = np.sqrt(mse)
    return rmse

In [18]:
# filling missing values
X_train_zeros = handle_nan(X_train, 'total_bedrooms', 'zero')
X_val_zeros = handle_nan(X_val, 'total_bedrooms', 'zero')

X_train_mean = handle_nan(X_train, 'total_bedrooms', 'mean')
X_val_mean = handle_nan(X_val, 'total_bedrooms', 'mean')

In [19]:
# For training set
Y_pred = linear_regression(X_train_zeros, Y_train)
rmse_train_zeros = rmse(Y_train, Y_pred)
print('RMSE for train set with zeros: ',round(rmse_train_zeros, 2) )

Y_pred = linear_regression(X_train_mean, Y_train)
rmse_train_mean= rmse(Y_train, Y_pred)
print('RMSE for train set with mean: ',round(rmse_train_mean, 2) )
#print('\n')

RMSE for train set with zeros:  0.34
RMSE for train set with mean:  0.34


In [20]:
# For validation set
Y_pred = linear_regression(X_val_zeros, Y_val)
rmse_val_zeros = rmse(Y_val, Y_pred)
print('RMSE for validation set with zeros: ', round(rmse_train_mean, 2) )

Y_pred = linear_regression(X_val_mean, Y_val)
rmse_val_mean = rmse(Y_val, Y_pred)
print('RMSE for validation set with mean: ',round(rmse_train_mean, 2) )

RMSE for validation set with zeros:  0.34
RMSE for validation set with mean:  0.34


Both are equally good

### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0.
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

Options:

- 0
- 0.000001
- 0.001
- 0.0001

In [21]:
def ridge_regression(X, y, r = 0.0):
    # adding ones in the dataset X
    X_0 = np.ones(X.shape[0])
    X = np.column_stack([X_0, X])

    XTX = X.T.dot(X)
    # add regularization term rI
    I = np.eye(XTX.shape[0])
    XTX_inverse = np.linalg.inv(XTX + r*I)
    w = XTX_inverse.dot(X.T).dot(y)

    Y_pred = X.dot(w)
    return Y_pred, w

In [22]:
# filling missing values with zeros
X_train = handle_nan(X_train, 'total_bedrooms', 'zero')
X_val = handle_nan(X_val, 'total_bedrooms', 'zero')


for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    Y_pred, _ = ridge_regression(X_val, Y_val, r=r)
    print('%10s' %r, round( rmse(Y_val, Y_pred), 2))

         0 0.34
     1e-06 0.34
    0.0001 0.34
     0.001 0.34
      0.01 0.34
       0.1 0.34
         1 0.35
         5 0.35
        10 0.35


### Question 5

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

What's the value of std?

- 0.5
- 0.05
- 0.005
- 0.0005


 Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different.
> If standard deviation of scores is low, then our model is *stable*.

In [24]:
def split_data(df, target_column, train_size = 0.6,
               val_size = 0.2, seed = 42, log_transform = True):

    if train_size + val_size >= 1.0:
        raise ValueError("Value larger then 1")

    n = len(df)
    n_train = int(train_size*n)
    n_val = int(val_size*n)

    # Shuffle data
    np.random.seed(seed)
    idx = np.arange(n)
    np.random.shuffle(idx)
    df_shuffled = df.iloc[idx]

    # Split data
    X_train = df_shuffled.iloc[:n_train].copy()
    X_val = df_shuffled.iloc[n_train:n_train + n_val].copy()
    X_test = df_shuffled.iloc[n_train + n_val:].copy()

    if log_transform:
        Y_train = np.log1p(X_train[target_column]).values
        Y_val = np.log1p(X_val[target_column]).values
        Y_test = np.log1p(X_test[target_column]).values
    else:
        Y_train = X_train[target_column].values
        Y_val = X_val[target_column].values
        Y_test = X_test[target_column].values

    del X_train[target_column]
    del X_val[target_column]
    del X_test[target_column]

    # Fill missing values with zeros
    X_train = handle_nan(X_train, 'total_bedrooms', 'zero')
    X_val = handle_nan(X_val, 'total_bedrooms', 'zero')
    X_test = handle_nan(X_test, 'total_bedrooms', 'zero')

    X = {'train':X_train, 'val':X_val, 'test':X_test}
    Y = {'train': Y_train,'val':Y_val,'test': Y_test}

    return X,Y

In [28]:
display(df_housing.head(2))

seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

errors = []
for seed in seeds:
    X,Y = split_data(df = df_housing, target_column= 'median_house_value', seed = seed)
    Y_pred = linear_regression(X['val'], Y['val'])
    error = rmse(Y['val'], Y_pred)

    print('%10s' %seed, round( error, 3) )
    errors.append( error )

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0


         0 0.337
         1 0.337
         2 0.337
         3 0.331
         4 0.337
         5 0.342
         6 0.334
         7 0.344
         8 0.35
         9 0.334


In [29]:
print('Std =', round(np.std(errors), 3))

Std = 0.005


### Question 6

* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with `r=0.001`.
* What's the RMSE on the test dataset?

Options:

- 0.13
- 0.23
- 0.33
- 0.43

In [31]:
# Split data and fill missing values with zeros
X,Y = split_data(df = df_housing, target_column= 'median_house_value', seed = 9)

In [32]:
# Combine train and validation
X_train = np.concatenate([ X['train'], X['val']])
Y_train = np.concatenate([ Y['train'], Y['val']])

In [33]:
# Train model on train and validation and use in test set
_, w = ridge_regression(X_train, Y_train, r = 0.001)

In [34]:
# Regression with the trained weights
Y_pred = w[0] + X['test'].dot(w[1:])

In [35]:
print('RMSE on test set = ', round( rmse(Y['test'], Y_pred), 2))

RMSE on test set =  0.33
